<a href="https://colab.research.google.com/github/janinerottmann/ITM22/blob/main/%C3%9Cbung5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IT Management \ Janine Rottmann \ Lehrstuhl für Wirtschaftsinformatik und Systementwicklung

# Simulating a Queueing System in Python

We all have visited a bank at some point in our life, and we are familiar with how banks operate. Customers enter, wait in a queue for their number to be called out, get service from the teller, and finally leave. This is a queueing system, and we encounter many queueing systems in our day to day lives, from grocery stores to amusement parks they’re everywhere. And that’s why we must try and make them as efficient as possible. There is a lot of randomness involved in these systems, which can cause huge delays, result in long queues, reduce efficiency, and even monetary loss. The randomness can be addressed by developing a discrete event simulation model, this can be extremely helpful in improving the operational efficiency, by analyzing key performance measures. In this project, we are simulating a queueing system for a bank and analyze its performance.

***Let’s consider a bank that has two tellers. Customers arrive at the bank about every 3 minutes on average according to a Poisson process. This rate of arrival is assumed in this case but should be modeled from actual data to get accurate results. They wait in a single line for an idle teller. This type of system is referred to as a M/M/2 queueing system. The average time it takes to serve a customer is 1.2 minutes by the first teller and 1.5 minutes by the second teller. The service times are assumed to be exponential here. When a customer enters the bank and both tellers are idle, they choose either one with equal probabilities. If a customer enters the bank and there are four people waiting in the line, they will leave the bank with probability 50%. If a customer enters the bank and there are five or more people waiting in the line, they will leave the bank with probability 90%.***

![alt text](https://github.com/janinerottmann/ITM22/blob/3be8280564df2db5d7f9089ffb488fea113cd87b/data/process_flow.png?raw=true)


# Building the Simulation Environment

The main components of a simulation model are:

* **State Variables**: describe the system at a particular time
* **Simulation Clock**: Keeps track of time
* **Statistical Counters**: Variables for storing statistical info about performance parameters
* **Initialization Routine**: A subprogram or class that initializes the model at time 0
* **Timing Routine**: A subprogram or a class that determines the next event
* **Event Routine**: A subprogram or a class that updates the system when a particular event occurs

In [1]:
# import libraries
import numpy as np
import pandas as pd

In [2]:
class Bank_Simulation:

    # we start by defining variables and initializing them in the init function, 
    # inside the main class. The variable defined below are state variables as 
    # well as statistical counters.
    # The key variables that tell us about the performance of the system are 
    # average wait time, utilization of servers, number of people waiting in line, 
    # and lost customers, some of which are directly calculated and others derived.
    def __init__(self): 
        self.clock=0.0                      #simulation clock
        self.num_arrivals=0                 #total number of arrivals
        self.t_arrival=self.gen_int_arr()   #time of next arrival
        self.t_departure1=float('inf')      #departure time from server 1
        self.t_departure2=float('inf')      #departure time from server 2
        self.dep_sum1=0                     #Sum of service times by teller 1
        self.dep_sum2=0                     #Sum of service times by teller 2
        self.state_T1=0                     #current state of server1 (binary)
        self.state_T2=0                     #current state of server2 (binary)
        self.total_wait_time=0.0            #total wait time
        self.num_in_q=0                     #current number in queue
        self.number_in_queue=0              #customers who had to wait in line(counter)
        self.num_in_system=0                #current number of customers in system
        self.num_of_departures1=0           #number of customers served by teller 1  
        self.num_of_departures2=0           #number of customers served by teller 2 
        self.lost_customers=0               #customers who left without service

    # The timing routine decides which event occurs next by comparing the 
    # scheduled time of events and advances the simulation clock to the 
    # respective event. Initially, the departure events are scheduled to occur 
    # at time infinity(since there are no customers), which guarantees that the 
    # first event will be an arrival event.
    def time_adv(self):   
        #determine time of next event                                                    
        t_next_event=min(self.t_arrival,self.t_departure1,self.t_departure2)  
        self.total_wait_time += (self.num_in_q*(t_next_event-self.clock))
        self.clock=t_next_event
                
        if self.t_arrival<self.t_departure1 and self.t_arrival<self.t_departure2:
            self.arrival()
        elif self.t_departure1<self.t_arrival and self.t_departure1<self.t_departure2:
            self.teller1()
        else:
            self.teller2()

    # As per the problem stated above, an arrival event can have multiple outcomes, 
    # which have been highlighted in the chart below.
    # The outcome of the arrival event is decided by the number of customers in 
    # the queue and the state of the servers. For every outcome, statistical 
    # counters are updated, and the next event is scheduled.
    def arrival(self):              
        self.num_arrivals += 1
        self.num_in_system += 1

        #schedule next departure or arrival depending on state of servers
        if self.num_in_q == 0:                              
            if self.state_T1==1 and self.state_T2==1:
                self.num_in_q+=1
                self.number_in_queue+=1
                self.t_arrival=self.clock+self.gen_int_arr()
                
                
            elif self.state_T1==0 and self.state_T2==0:
                
                if np.random.choice([0,1])==1:
                    self.state_T1=1
                    self.dep1= self.gen_service_time_teller1()
                    self.dep_sum1 += self.dep1
                    self.t_departure1=self.clock + self.dep1
                    self.t_arrival=self.clock+self.gen_int_arr()

                else:
                    self.state_T2=1
                    self.dep2= self.gen_service_time_teller2()
                    self.dep_sum2 += self.dep2
                    self.t_departure2=self.clock + self.dep2
                    self.t_arrival=self.clock+self.gen_int_arr()

            #if server 2 is busy customer goes to server 1
            elif self.state_T1==0 and self.state_T2 ==1:    
                self.dep1= self.gen_service_time_teller1()
                self.dep_sum1 += self.dep1
                self.t_departure1=self.clock + self.dep1
                self.t_arrival=self.clock+self.gen_int_arr()
                self.state_T1=1
            
            #otherwise customer goes to server 2
            else:                                           
                self.dep2= self.gen_service_time_teller2()
                self.dep_sum2 += self.dep2
                self.t_departure2=self.clock + self.dep2
                self.t_arrival=self.clock+self.gen_int_arr()
                self.state_T2=1
        
        #if queue length is less than 4 generate next arrival and make customer join queue
        elif self.num_in_q < 4 and self.num_in_q >= 1:
            self.num_in_q+=1
            self.number_in_queue+=1                             
            self.t_arrival=self.clock + self.gen_int_arr()
        
        #if queue length is 4 equal prob to leave or stay
        elif self.num_in_q == 4:                             
            if np.random.choice([0,1])==0: 
                self.num_in_q+=1 
                self.number_in_queue+=1                 
                self.t_arrival=self.clock + self.gen_int_arr()
            else:
                self.lost_customers+=1
                
        #if queue length is more than 5 90% chance of leaving 
        elif self.num_in_q >= 5:                            
            if np.random.choice([0,1],p=[0.1,0.9])==0:
                self.t_arrival=self.clock+self.gen_int_arr()
                self.num_in_q+=1 
                self.number_in_queue+=1 
            else:
                self.lost_customers+=1      

    # A departure event occurs when the timing routine identifies any of the two 
    # departure events to be scheduled next.

    # Departure from server 1
    def teller1(self):            
        self.num_of_departures1 += 1
        self.num_in_system -= 1 
        if self.num_in_q>0:
            self.dep1= self.gen_service_time_teller1()
            self.dep_sum1 += self.dep1
            self.t_departure1=self.clock + self.dep1
            self.num_in_q-=1
        else:
            self.t_departure1=float('inf') 
            self.state_T1=0                  
    # Departure from server 2
    def teller2(self):              
        self.num_of_departures2 += 1
        self.num_in_system -= 1
        if self.num_in_q>0:
            self.dep2= self.gen_service_time_teller2()
            self.dep_sum2 += self.dep2
            self.t_departure2=self.clock + self.dep2
            self.num_in_q-=1
        else:
            self.t_departure2=float('inf')
            self.state_T2=0
    
    # The arrival and service times are generated using inverse transform sampling. 
    # These random number generator functions are called in the arrival and departure 
    # functions to generate random arrivals and service times.

    # Function to generate arrival times using inverse transform
    def gen_int_arr(self):                                             
        return (-np.log(1-(np.random.uniform(low=0.0,high=1.0))) * 3)
    
    # Function to generate service time for teller 1 using inverse transform
    def gen_service_time_teller1(self):                              
        return (-np.log(1-(np.random.uniform(low=0.0,high=1.0))) * 1.2)
    
    # Function to generate service time for teller 1 using inverse transform
    def gen_service_time_teller2(self):                                
        return (-np.log(1-(np.random.uniform(low=0.0,high=1.0))) * 1.5)

# Running the Simulation

Having built the simulation environment, we can run the simulation. We run the simulation over a period of one year (364 replications). Each workday starts at 8am and ends at 4pm. Data is collected and stored in a pandas data frame for further analysis. The data frame may be exported as an excel workbook.

In [3]:
# create simulation environment
s = Bank_Simulation()

# create a pandas data frame to report results
df = pd.DataFrame(columns=['Num_arrivals',
                           'Average_interarrival_time',
                           'Average_service_time_teller1',
                           'Average_service_time_teller 2',
                           'Utilization_teller_1',
                           'Utilization_teller_2',
                           'People_who_had_to_wait_in_line',
                           'Total_average_wait_time',
                           'Lost_Customers'])

# rund simulation for 364 days
for i in range(364):
    
    # set random seed
    np.random.seed(i)
    s.__init__()
    
    # simulate 8 hours (8 * 60 = 480)
    while s.clock <= 480 :
        s.time_adv() 
    
    # store results of each simulation run
    a = pd.Series([s.num_arrivals,
                   s.clock/s.num_arrivals,
                    s.dep_sum1/s.num_of_departures1,
                    s.dep_sum2/s.num_of_departures2,
                    s.dep_sum1/s.clock,
                    s.dep_sum2/s.clock,
                    s.number_in_queue,
                    s.total_wait_time,
                    s.lost_customers],index=df.columns)
    df = df.append(a,ignore_index=True)

# Exercises

Now, we can derive performance measures from these results to then analyze them and use them in improving efficiency, reducing costs, allocating resources, etc.
Analyze the resulting eventlog and answer the following questions:

* **Exercise 2.1: What was the average utilization of teller 1 and 2 last year?** 

In [4]:
# Please enter your code here.

* **Exercise 2.2: On average, how many customers had to wait in line to be served?**

In [5]:
# Please enter your code here.

* **Exercise 2.3: How long on average does a customer have to wait to be served (in minutes)?**

In [6]:
# Please enter your code here.

* **Exercise 2.4: How many customers visited the bank last year?**

In [7]:
# Please enter your code here.

* **Exercise 2.5: What percentage of customers did the bank lose due to long waiting times (rounded to two decimal places)?**

In [8]:
# Please enter your code here.

Simulation can be used as a crucial decision-making tool in many industries, from manufacturing to service and even biology. This project is just a small example of the endless possibilities of simulation. There are many Softwares and resources out there to model complex systems and simulate them with ease. However, the goal of this project was to get a fundamental understanding of how a discrete event simulation works and its use as a decision-making tool.